# Expanding American Restaurant

In [2]:
!conda install beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5.1-py36h~ --> pkgs/main::certifi-2020.4.5.1-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!pip install geocoder

In [4]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi              pkgs/main::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl              pkgs/main::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Libraries imported.


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='wikitable sortable')
tbody = table.find('tbody')
trs = tbody.find_all('tr')


In [6]:
postal_codes = []
boroughs = []
neighborhoods = []

td_i = 0;

for tr in trs:
    tds = tr.find_all('td')
    
    postal_code = ''
    borough = ''
    neighborhood = ''
    
    for td in tds:
        
        if td_i == 0:
            postal_code = td.text.rstrip('\n')
        elif td_i == 1:
            borough = td.text.rstrip('\n')
        elif td_i == 2:
            neighborhood = td.text.rstrip('\n')
            
        td_i = td_i + 1
        
    if borough != 'Not assigned' and borough != '':
            postal_codes.append(postal_code)
            boroughs.append(borough)
            neighborhoods.append(neighborhood)
        
    td_i = 0
    
np_data = np.array([postal_codes, boroughs, neighborhoods]).transpose()
df = pd.DataFrame(data=np_data, columns=['Postal Code', 'Borough', 'Neighborhood'])
df.shape

(103, 3)

In [7]:
lldf = pd.read_csv('http://cocl.us/Geospatial_data')

In [112]:
df['Latitude'] = 0
df['Longitude'] = 0
df = df.astype({'Latitude': float, 'Longitude': float})

In [9]:
for index, row in df.iterrows():
    
    lon_lat_r = lldf.loc[lldf['Postal Code'] == row['Postal Code']]
    df.loc[index, 'Latitude'] = lon_lat_r[['Latitude']].iloc[0].values[0]
    df.loc[index, 'Longitude'] = lon_lat_r[['Longitude']].iloc[0].values[0]
    

In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [113]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto

## Central Toronto

In [36]:
central_toronto_data = df[df['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [14]:
address = 'Central Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


In [115]:
# create map of Central Toronto using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto_data['Latitude'], central_toronto_data['Longitude'], central_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

In [16]:
CLIENT_ID = 'ZLSQS0RYGDKURRJARHQI03WTSPFABEREK2S5D4BTRKMIKQEZ' # your Foursquare ID
CLIENT_SECRET = 'Q45QDIF1TATI4OQW1YPBIT4VDHOAARBRJYCQRP4N1HCMXYUC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZLSQS0RYGDKURRJARHQI03WTSPFABEREK2S5D4BTRKMIKQEZ
CLIENT_SECRET:Q45QDIF1TATI4OQW1YPBIT4VDHOAARBRJYCQRP4N1HCMXYUC


In [17]:
central_toronto_data.loc[0, 'Neighborhood']

'Lawrence Park'

In [18]:
neighborhood_latitude = central_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Get the top 500 venues that are in Lawrence Park within a radius of 1000 meters.

In [75]:
# type your answer here

LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [77]:
results = requests.get(url).json()

In [78]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


## Explore neighborhood in Central Toronto

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
central_toronto_venues = getNearbyVenues(names=central_toronto_data['Neighborhood'],
                                   latitudes=central_toronto_data['Latitude'],
                                   longitudes=central_toronto_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [94]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(557, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.72802,-79.38879,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.72802,-79.38879,Granite Club,43.733043,-79.381986,Gym / Fitness Center
2,Lawrence Park,43.72802,-79.38879,Tim Hortons,43.727324,-79.379563,Coffee Shop
3,Lawrence Park,43.72802,-79.38879,Granite Club President's Lounge,43.733005,-79.382059,Café
4,Lawrence Park,43.72802,-79.38879,Glendon Bookstore,43.727024,-79.378976,Bookstore


In [175]:
df_venue = central_toronto_venues[['Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']]

In [176]:
df_venue.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Granite Club,43.733043,-79.381986,Gym / Fitness Center
2,Tim Hortons,43.727324,-79.379563,Coffee Shop
3,Granite Club President's Lounge,43.733005,-79.382059,Café
4,Glendon Bookstore,43.727024,-79.378976,Bookstore


In [177]:
from sklearn.preprocessing import StandardScaler
X = df_venue.values[:,1:3]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
#Clus_dataSet = X

clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_

df_venue['segment'] = labels
labels

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2,
       1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [178]:
df_venue.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,segment
0,Lawrence Park Ravine,43.726963,-79.394382,Park,2
1,Granite Club,43.733043,-79.381986,Gym / Fitness Center,2
2,Tim Hortons,43.727324,-79.379563,Coffee Shop,2
3,Granite Club President's Lounge,43.733005,-79.382059,Café,2
4,Glendon Bookstore,43.727024,-79.378976,Bookstore,2


## 3 Segment of Venue in Central Toronto

In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_venue['Venue Latitude'], df_venue['Venue Longitude'], df_venue['Venue'], df_venue['segment']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Get the most strategic cluster to expand the restaurant
because of the office building is nothing on the data, so I assume that Bank our market, where many executives there who prefer restaurants which can provide fast service with still high-quality food

### Bank per segment

In [210]:
df_venue[(df_venue['Venue Category'] == 'Bank')].groupby('segment').agg(['count'])

,Venue,Venue Latitude,Venue Longitude,Venue Category
,count,count,count,count
segment,,,,
0,4,4,4,4
1,2,2,2,2
2,4,4,4,4


### American restaurant per segment

In [211]:
df_venue[(df_venue['Venue Category'] == 'American Restaurant')].groupby('segment').agg(['count'])

,Venue,Venue Latitude,Venue Longitude,Venue Category
,count,count,count,count
segment,,,,
0,2,2,2,2
1,2,2,2,2


## Conclusion
_From this data, we can conclude that segment 2 is the best segment to build the restaurant because the number of bank is 4 whereas the number of american restaurant is 0, therefore difference is 4. It's better compared to other segments, which are 2 and 0 difference for segment 0 and 1 respectively_

In [208]:
k_means.cluster_centers_

array([[ 43.68816197, -79.39330718],
       [ 43.67319472, -79.40594706],
       [ 43.70810928, -79.39796236]])

Since the best cluster is cluster 2, then the chosen coordinate for the new restaurant is 43.70810928, -79.39796236